### Imports

In [ ]:
import requests
import os
import json
from bs4 import BeautifulSoup

### Utils

In [ ]:
def extract(ancestor, selector, attribute=None, index=False):
    if index:
        if attribute:
            return [tag[attribute] for tag in (ancestor.select(selector))]
        return [tag.get_text() for tag in (ancestor.select(selector))]
    if attribute:
        try:
            return str(ancestor.select_one(selector)[attribute].get_text())
        except TypeError:
            return None
    try:
        return str(ancestor.select_one(selector).get_text())
    except AttributeError:
        return None

### Exraction of opinions

In [ ]:
with open('./cookie.json', 'r') as file:
    headers = json.load(file)

In [ ]:
product_id = '84514582'
next_page = f'https://www.ceneo.pl/{product_id}#tab=reviews'
all_opinions = []
while next_page:
    response = requests.get(next_page, headers=headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        for opinion in opinions:
            single_opinion = {}
            single_opinion["option-id"] = opinion["data-entry-id"]
            single_opinion["author"] = extract(opinion, "span.user-post__author-name")
            single_opinion["recommendation"] = extract(opinion, "span.user-post__author-recomendation > em")
            single_opinion["stars"] = extract(opinion, "span.user-post__score-count")
            single_opinion["content"] = extract(opinion, "div.user-post__text")
            single_opinion["pros"] = extract(opinion, "div.review-feature__item--positive", None, True)
            single_opinion["cons"] = extract(opinion, "div.review-feature__item--negative", None, True)
            single_opinion["vote_yes"] = extract(opinion, "button.vote-yes", "data-total-vote")
            single_opinion["vote_no"] = extract(opinion, "button.vote-no", "data-total-vote")
            single_opinion["published_at"] = extract(opinion, "span.user-post__published > time:nth-child(1)", "datetime")
            single_opinion["purchased_at"] = extract(opinion, "span.user-post__published > time:nth-child(2)", "datetime")
            all_opinions.append(single_opinion)
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except AttributeError:
            next_page = None
        except TypeError:
            next_page = None
print(all_opinions)

In [ ]:
if not os.path.exists('./opinions'):
    os.mkdir('./opinions')
with open(f"./opinions/{product_id}.json", 'w', encoding='UTF-8') as file:
    json.dump(all_opinions, file, ensure_ascii=False, indent=4)